# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
#qk.IBMQ.save_account("66718f8f8aef22bcb6ebe86ad94a11f1fd1f4c55100829bb13f16e6b448e0a1ec6d09c459d738f58d0cbd8398a2a1f5e185a4706a61b6f896a5ce2983e136429", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibmq_santiago")

RequestsApiError: "HTTPSConnectionPool(host='auth.quantum-computing.ibm.com', port=443): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ffa9aabf350>: Failed to establish a new connection: [Errno -2] Name or service not known'))"

## Quantum State Tomography

### Two Qubits

In [2]:
n = 2

circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)

basis_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit") for i in range(6**n)]
state_input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "density") for i in range(6**n)]

qsr_list = []
for basis in basis_list:
    circuit = qk.circuit.QuantumCircuit.compose(basis, circuit_target)
    qsr_list.append(StateTomography(circuit))
    
result_list = [qsr.run(backend, shots=20000).block_for_results() for qsr in tqdm(qsr_list)]
state_target_list = [np.array(result.analysis_results("state").value) for result in result_list]

pickle.dump([state_input_list, state_target_list], open("twoQubits_singleBlock.p", "wb"))

NameError: name 'StateTomography' is not defined

### Three Qubits

In [5]:
n = 3

circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.h(1)
circuit_target.s(2)
circuit_target.cnot(1,2)

basis_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit") for i in range(6**n)]
state_input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "density") for i in range(6**n)]

qsr_list = []
for basis in basis_list:
    circuit = qk.circuit.QuantumCircuit.compose(basis, circuit_target)
    qsr_list.append(StateTomography(circuit))
    
result_list = [qsr.run(backend, shots=20000).block_for_results() for qsr in tqdm(qsr_list)]
state_target_list = [np.array(result.analysis_results("state").value) for result in result_list]

pickle.dump([state_input_list, state_target_list], open("threeQubits_singleBlock.p", "wb"))

## Pauli String Expectation Values

### Two Qubits

In [4]:
backend = AerSimulator()
n = 3
N = 1000
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.h(1)
circuit_target.s(2)
circuit_target.cnot(1,2)

np.random.seed(42)
random.seed(42)

input_list = []
circuit_list = []
for i in range(N):
    
    index = np.random.randint(0, 6**n)
    config = numberToBase(index, 6, n)
    state = prepare_input(config)
    state_circuit = prepare_input(config, return_mode = "circuit")

    index = np.random.randint(1, 4**n)
    config = numberToBase(index, 4, n)
    observable = pauli_observable(config)
    observable_circuit = pauli_observable(config, return_mode = "circuit")
    
    
    input_list.append([state, observable])
    circuit = state_circuit
    circuit = circuit.compose(circuit_target)
    circuit.add_register(observable_circuit.cregs[0])
    circuit = circuit.compose(observable_circuit)
    
    circuit_list.append(circuit)

In [5]:
result_list = qk.execute(circuit_list, backend, shots = 10000).result()
counts_list = [result_list.get_counts(circuit) for circuit in circuit_list]

In [7]:
expectation_list = [expected_parity(counts) for counts in counts_list]
data = [input_list, expectation_list]

pickle.dump(data, open("..\..\data\threeQubits_expectation.p", "wb"))